In [192]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
df = pd.read_csv('AWD-J.txt',sep="\t")

df.sample(5)

,Word,Org,Score,Deviation,Std,PoS
1055,クローズアップ現代,クローズアップ現代,1.50,65.05,0.92,名詞
9593,hit,hit,2.40,46.47,1.33,名詞
11035,歴代,歴代,2.53,43.71,1.50,名詞
9991,天神,天神,2.43,45.78,1.15,名詞
2308,新番組,新番組,1.70,60.92,0.78,名詞


In [193]:
import math


df["integer_part"] = df["Score"].apply(lambda x: math.modf(x)[1])
df.sample(5)

,Word,Org,Score,Deviation,Std,PoS,integer_part
357,ドーナツ,ドーナツ,1.33,68.49,0.60,名詞,1.0
6736,まんが,まんが,2.17,51.28,1.27,名詞,2.0
10158,北斗,北斗,2.47,45.09,1.06,名詞,2.0
14246,万が一,万が一,2.97,34.77,1.30,名詞,2.0
435,豚,豚,1.33,68.49,0.54,名詞,1.0


In [194]:
unique_integer_paths = df["integer_part"].unique()
samples_by_integer_part = []

for score in unique_integer_paths:
    samples_by_integer_part.append(df[df["integer_part"] == score].sample(1,replace=True))

samples_by_integer_part

[     Word  Org  Score  Deviation   Std PoS  integer_part
 1843  あずき  あずき   1.63      62.29  1.08  名詞           1.0,
      Word Org  Score  Deviation  Std PoS  integer_part
 5391   放映  放映   2.03      54.04  1.2  名詞           2.0,
       Word Org  Score  Deviation   Std PoS  integer_part
 14989   ヤバ  ヤバ   3.27      28.57  1.03  名詞           3.0,
        Word   Org  Score  Deviation   Std PoS  integer_part
 15219  ホンワカ  ホンワカ   4.13      10.68  1.06  名詞           4.0]

In [195]:
def df_to_prompt_example(df):
    return f"{df["Word"].iloc[0]}: f{df["Score"].iloc[0]}\n"


In [196]:
prompt_example = ""
for df in samples_by_integer_part:
    prompt_example += df_to_prompt_example(df)
prompt = f"""
あなたは言葉の抽象度を評価するアシスタントです。
ユーザが提供したことの抽象度を判断してください。
以下を参考にしてください。
{prompt_example}

かならず以下のフォーマットに従ってください。
word: score
"""

prompt

'\nあなたは言葉の抽象度を評価するアシスタントです。\nユーザが提供したことの抽象度を判断してください。\n以下を参考にしてください。\nあずき: f1.63\n放映: f2.03\nヤバ: f3.27\nホンワカ: f4.13\n\n\nかならず以下のフォーマットに従ってください。\nword: score\n'

In [197]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content":"積極的なコミュニケーション,職位の壁を超えたコミュにケーション"}])

response.choices[0].message.content



'積極的なコミュニケーション: f4.56\n職位の壁を超えたコミュニケーション: f4.72'